In [ ]:
from tensorflow import keras
import tensorflow as tf
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
#read dataset

names = ["Vinit","Shourish","Mihir","Parth"]
# gestures = ["up","right","select","stationary1","stationary2"] #except downleft
# gestures = ["up","right","down","select","stationary1","stationary2"] #with down
gestures = ["up","right","left","down","select","stationary1","stationary2"] #allgest alldata
# gestures = ["up","right","left","down","select","stationary1","stationary2"]

images_pername_pergest  = 100

N = 2*100*len(gestures)+2*200*len(gestures)
dataset = np.zeros((N, 250, 170, 3), np.uint8)

d_index = 0
labels = np.zeros((N,1), np.int8)

for gest in gestures:
    for name in names:
        if (name == "Vinit"and gest != "left") or name =="Mihir":
            for i in range(100):
                img  = cv2.imread(fr"E:\USERS\VINIT\Desktop\MainProject\datasetColor\{name}\{gest}\{name}_{i}.png")
                dataset[d_index] = img
                labels[d_index] = gestures.index(gest)
                d_index += 1
        elif name=="Parth" or name=="Shourish":
            for i in range(100):
                img  = cv2.imread(fr"E:\USERS\VINIT\Desktop\MainProject\datasetColor\{name}\{gest}\{name}_{i}.png")
                dataset[d_index] = img
                labels[d_index] = gestures.index(gest)
                d_index += 1


In [ ]:
#Normalisation and Dividing the set
dataset = dataset/255
X_train,X_test,y_train,y_test=train_test_split(dataset,labels,test_size=0.10,shuffle=True)


In [ ]:
# name = "Vinit"
# gest = "up"
# i = 100
# img  = cv2.imread(fr"E:\USERS\VINIT\Desktop\MainProject\datasetColor\{name}\{gest}\{name}_{i}.png")
# # type(img)
# import matplotlib.pyplot as plt
# plt.imshow(img)

## Model Architecture & Training

In [ ]:
#Model2

model = keras.Sequential([
    keras.layers.AveragePooling2D(6,3, input_shape=(250,170,3)),
    keras.layers.Conv2D(512, 3, activation='relu'),
    keras.layers.Conv2D(256, 3, activation='relu'),
    keras.layers.Conv2D(128, 3, activation='relu'),
    keras.layers.Conv2D(64, 3, activation='relu'),
    keras.layers.Conv2D(32, 3, activation='relu'),
    keras.layers.MaxPool2D(2,2),
    keras.layers.Dropout(0.5),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(gestures), activation='softmax')
 ])

model.compile(optimizer='adam',loss=keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

model.fit(X_train, y_train, epochs=10, batch_size=20)
# tf.keras.utils.plot_model(model, to_file="image.png", show_shapes=True)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
a = dataset[1]
a = a.reshape(1,250,170,3)
model.predict_classes(a)

In [ ]:
import cv2
import numpy as np


def main():
    cap=cv2.VideoCapture(0)
    mfound=False
    one=False
    i=0

    while True:
        ret, frame = cap.read()
        roi=frame[150:400,450:620]/255
        cv2.rectangle(frame, (450,150), (620,400), (255,255,255))

        
        prediction=model.predict_classes(roi.reshape(1,250,170,3))    
        font =cv2.FONT_HERSHEY_COMPLEX
        cv2.putText(frame, "THIS IS "+gestures[prediction[0]], (0,200), font, 2, (255,255,0), 2)
        cv2.imshow("frame", frame)

        inp=cv2.waitKey(1)
        if inp==27:
            cap.release()
            cv2.destroyAllWindows()
            break
            
main()

In [ ]:
model.save("colour.h5")